## Cheat Sheet for NBA games on Jan 22 2025

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from teamNames import team_name_to_abbreviation
import json
import http.client

In [18]:
all_games_playing = pd.read_csv('januarySchedule.csv')
df = pd.read_csv('prizepicks_nba_1-22-2025.csv') 
df

,Player,Team,Stat,Line
0,Dyson Daniels,ATL,Pts+Rebs,18.5
1,Dyson Daniels,ATL,Rebs+Asts,9.0
2,Dyson Daniels,ATL,Rebounds,5.0
3,Dyson Daniels,ATL,Pts+Asts,17.5
4,Dyson Daniels,ATL,Pts+Rebs+Asts,22.5
...,...,...,...,...
1809,James Harden,LAC,Assists,9.5
1810,James Harden,LAC,3-PT Made,1.5
1811,James Harden,LAC,Points,19.5
1812,James Harden,LAC,Points,24.5


In [19]:
# in the all games_playing dataframe the first collumn in the "Date". Find all of the games that are playing today. the format of the date in the collumn is: Wed Jan 1 2025
today = pd.to_datetime('today').strftime('%a %b %d %Y')
today_games = all_games_playing[all_games_playing['Date'] == today]
# gets the teams playing today in abbreviation form
teamsToday = today_games[['Visitor/Neutral','Home/Neutral']].copy()
teamsToday['Visitor/Neutral'] = teamsToday['Visitor/Neutral'].apply(lambda x: team_name_to_abbreviation(x))
teamsToday['Home/Neutral'] = teamsToday['Home/Neutral'].apply(lambda x: team_name_to_abbreviation(x))
teamsToday

,Visitor/Neutral,Home/Neutral
153,DET,ATL
154,PHX,BKN
155,MIN,DAL
156,CLE,HOU
157,CHA,MEM
158,MIL,NOP
159,UTH,OKC
160,GSW,SAC
161,BOS,LAC


In [20]:
# Iterate through each row in teamsToday
for index, row in teamsToday.iterrows():
    visitor = row["Visitor/Neutral"]
    home = row["Home/Neutral"]
    matchup_key = f"{visitor}_vs_{home}"  # Use a valid variable name format
    
    # Filter the df DataFrame for players from the visitor and home teams
    visitor_data = df[df["Team"] == visitor]
    home_data = df[df["Team"] == home]
    
    # Combine the data for the matchup
    matchup_df = pd.concat([visitor_data, home_data], ignore_index=True)
    
    # Dynamically create a DataFrame variable for the matchup
    globals()[matchup_key] = matchup_df

In [21]:
import json

import http.client

conn = http.client.HTTPSConnection("tank01-fantasy-stats.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "dfd93611e1msh1e23e65c82ec5fep17ce07jsn8b6e2418710f",
    'x-rapidapi-host': "tank01-fantasy-stats.p.rapidapi.com"
}

conn.request("GET", "/getNBAPlayerList", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8")) # put this in a data frame called playerList

# Convert the JSON data to a dictionary
player_data = json.loads(data.decode("utf-8"))
# Convert the 'body' dictionary to a pandas DataFrame 
playerList = pd.DataFrame(player_data['body'])


{"statusCode": 200, "body": [{"pos": "PF", "playerID": "28368569499", "team": "", "longName": "Aric Holman", "teamID": ""}, {"pos": "PG", "playerID": "94914298027", "team": "CHA", "longName": "LaMelo Ball", "teamID": "4"}, {"pos": "PG", "playerID": "94914479047", "team": "MIA", "longName": "Tyler Herro", "teamID": "16"}, {"pos": "PG", "playerID": "28798327499", "team": "ORL", "longName": "Devin Cannady", "teamID": "22"}, {"pos": "PF", "playerID": "28456813032", "team": "DAL", "longName": "Markieff Morris", "teamID": "7"}, {"pos": "C", "playerID": "94404412027", "team": "TOR", "longName": "Orlando Robinson", "teamID": "28"}, {"pos": "PF", "playerID": "94614279027", "team": "ORL", "longName": "Franz Wagner", "teamID": "22"}, {"pos": "SF", "playerID": "941942737289", "team": "MEM", "longName": "Tarik Biberovic", "teamID": "15"}, {"pos": "SF", "playerID": "28038159069", "team": "MIN", "longName": "Justin Jackson", "teamID": "18"}, {"pos": "PG", "playerID": "28598307499", "team": "HOU", "lo

In [22]:
# make me a function that takes in a player name and returns the player id
def get_player_id(player_name):
    # Filter the DataFrame to find the player
    player_row = playerList[playerList['longName'] == player_name]
    
    if not player_row.empty:
        # Return the player ID
        return player_row.iloc[0]['playerID']
    else:
        return None

In [23]:
import http.client
import json


# Function to fetch the last N games for a player
def get_last_n_games(player_id, season=2024, n=15):
    conn = http.client.HTTPSConnection("tank01-fantasy-stats.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "dfd93611e1msh1e23e65c82ec5fep17ce07jsn8b6e2418710f",  # Replace with your API key
        'x-rapidapi-host': "tank01-fantasy-stats.p.rapidapi.com"
    }
    # Make the API request
    conn.request("GET", f"/getNBAGamesForPlayer?playerID={player_id}&season={season}&fantasyPoints=true&pts=1&reb=1.25&stl=3&blk=3&ast=1.5&TOV=-1&mins=0&doubleDouble=0&tripleDouble=0&quadDouble=0", headers=headers)
    res = conn.getresponse()
    data = res.read()
    json_data = json.loads(data.decode("utf-8"))
    
    # Return the games data
    if json_data.get("statusCode") == 200:
        games_data = json_data.get("body", {})
        # Sort games by date (assuming gameID contains date) and return the last N games
        sorted_games = sorted(games_data.items(), key=lambda x: x[0], reverse=True)[:n]
        return dict(sorted_games)
    else:
        print(f"Error fetching games for player ID {player_id}: {json_data.get('message', 'Unknown error')}")
        return {}

# Function to calculate hit rate for each prop
def calculate_hitrate(matchup_df, season=2025):
    # Add new columns for hit rates
    matchup_df['hitrate_last5'] = None
    matchup_df['hitrate_last10'] = None
    matchup_df['hitrate_last15'] = None

    # Group the DataFrame by player to avoid redundant API calls
    grouped_players = matchup_df.groupby('Player')

    for player_name, group in grouped_players:
        # Get player ID
        player_id = get_player_id(player_name)
        if not player_id:
            print(f"Player ID not found for {player_name}")
            continue
        
        # Fetch last 15 games (we'll use subsets for 5 and 10 games)
        games_data = get_last_n_games(player_id, season, n=15)
        if not games_data:
            print(f"No games found for {player_name}")
            continue
        
        # Extract subsets for last 5 and 10 games
        games_last5 = dict(list(games_data.items())[:5])
        games_last10 = dict(list(games_data.items())[:10])
        games_last15 = games_data  # Already has last 15 games

        # Calculate hit rates for each prop in the group
        for index, row in group.iterrows():
            stat = row['Stat']
            line = row['Line']

            def calculate_hit_count(games_data):
                hit_count = 0
                for game_id, game_stats in games_data.items():
                    if stat == "Pts+Asts":
                        player_stat = float(game_stats.get('pts', 0)) + float(game_stats.get('ast', 0))
                    elif stat == "Rebs+Asts":
                        player_stat = float(game_stats.get('reb', 0)) + float(game_stats.get('ast', 0))
                    elif stat == "Points":
                        player_stat = float(game_stats.get('pts', 0))
                    elif stat == "Rebounds":
                        player_stat = float(game_stats.get('reb', 0))
                    elif stat == "Assists":
                        player_stat = float(game_stats.get('ast', 0))
                    elif stat == "Steals":
                        player_stat = float(game_stats.get('stl', 0))
                    elif stat == "Blocks":
                        player_stat = float(game_stats.get('blk', 0))
                    elif stat == "Blks+Stls":
                        player_stat = float(game_stats.get('blk', 0)) + float(game_stats.get('stl', 0))
                    elif stat == "Pts+Rebs":
                        player_stat = float(game_stats.get('pts', 0)) + float(game_stats.get('reb', 0))
                    elif stat == "Pts+Rebs+Asts":
                        player_stat = float(game_stats.get('pts', 0)) + float(game_stats.get('reb', 0)) + float(game_stats.get('ast', 0))
                    elif stat == "3-PT Made":
                        player_stat = float(game_stats.get('tptfgm', 0))
                    elif stat == "Blocked Shots":
                        player_stat = float(game_stats.get('blk', 0))
                    elif stat == "Turnovers":
                        player_stat = float(game_stats.get('TOV', 0))  # Corrected key for turnovers
                    elif stat == "FG Attempted":
                        player_stat = float(game_stats.get('fga', 0))
                    elif stat == "FG Made":
                        player_stat = float(game_stats.get('fgm', 0))
                    elif stat == "Defensive Rebounds":
                        player_stat = float(game_stats.get('DefReb', 0))
                    elif stat == "Offensive Rebounds":
                        player_stat = float(game_stats.get('OffReb', 0))
                    elif stat == "3-PT Attempted":
                        player_stat = float(game_stats.get('tptfga', 0))
                    else:
                        print(f"Unsupported stat: {stat}")
                        continue
                    
                    if player_stat >= line:
                        hit_count += 1
                return hit_count
            
            # Calculate hit rates for last 5, 10, and 15 games
            hitrate_last5 = calculate_hit_count(games_last5)
            hitrate_last10 = calculate_hit_count(games_last10)
            hitrate_last15 = calculate_hit_count(games_last15)
            
            # Update hit rates in the DataFrame
            matchup_df.at[index, 'hitrate_last5'] = f"{hitrate_last5}/5"
            matchup_df.at[index, 'hitrate_last10'] = f"{hitrate_last10}/10"
            matchup_df.at[index, 'hitrate_last15'] = f"{hitrate_last15}/15"
    
    return matchup_df

Change this list content to the games playing today. 

In [24]:
matchup_df = [DET_vs_ATL, PHX_vs_BKN, MIN_vs_DAL, CLE_vs_HOU, CHA_vs_MEM, MIL_vs_NOP, UTH_vs_OKC, GSW_vs_SAC, BOS_vs_LAC]

In [25]:
def calculate_hitrate_for_all_matchups(matchup_dfs, season=2025):
    """
    Applies hit rate calculation to all matchup DataFrames.

    Parameters:
        matchup_dfs (list of pd.DataFrame): List of DataFrames, each representing a matchup.
        season (int): The season year to fetch data for.

    Returns:
        list of pd.DataFrame: List of updated DataFrames with hit rates added.
    """
    updated_matchup_dfs = []

    for matchup_df in matchup_dfs:
        # Calculate hit rates for the current matchup DataFrame
        updated_df = calculate_hitrate(matchup_df, season)
        updated_matchup_dfs.append(updated_df)
    
    return updated_matchup_dfs

In [ ]:
updated_matchup_dfs = calculate_hitrate_for_all_matchups(matchup_df, season=2025)

In [ ]:
# renames the columns
for i in updated_matchup_dfs:
    i.rename(columns={'hitrate_last5': 'Last 5 games', 'hitrate_last10': 'Last 10 games', 'hitrate_last15': 'Last 15 games'}, inplace=True)


In [33]:
#if the last 5 games is 4/5 or 5/5 then make the background green. if it is 3/5 then make the background yellow. if it is 2/5 or less then make the background red
# if the last 10 games is 8/10 or 9/10 or 10/10 then make the background green. if it is 7/10 then make the background yellow. if it is 6/10 or less then make the background red
# if the last 15 games is 12/15 or 13/15 or 14/15 or 15/15 then make the background green. if it is 11/15 then make the background yellow. if it is 10/15 or less then make the background red
def highlight_hitrate(row):
    styles = [''] * len(row)
    
    last_5 = row['Last 5 games']
    last_10 = row['Last 10 games']
    last_15 = row['Last 15 games']
    
    if last_5 == "4/5" or last_5 == "5/5":
        styles[row.index.get_loc('Last 5 games')] = 'background-color: rgba(0, 255, 0, 0.3)'
    elif last_5 == "3/5":
        styles[row.index.get_loc('Last 5 games')] = 'background-color: rgba(255, 255, 0, 0.3)'
    elif last_5 in ["2/5", "1/5", "0/5"]:
        styles[row.index.get_loc('Last 5 games')] = 'background-color: rgba(255, 0, 0, 0.3)'
    
    if last_10 in ["8/10", "9/10", "10/10"]:
        styles[row.index.get_loc('Last 10 games')] = 'background-color: rgba(0, 255, 0, 0.3)'
    elif last_10 == "7/10":
        styles[row.index.get_loc('Last 10 games')] = 'background-color: rgba(255, 255, 0, 0.3)'
    elif last_10 in ["6/10", "5/10", "4/10", "3/10", "2/10", "1/10", "0/10"]:
        styles[row.index.get_loc('Last 10 games')] = 'background-color: rgba(255, 0, 0, 0.3)'
    
    if last_15 in ["12/15", "13/15", "14/15", "15/15"]:
        styles[row.index.get_loc('Last 15 games')] = 'background-color: rgba(0, 255, 0, 0.3)'
    elif last_15 == "11/15":
        styles[row.index.get_loc('Last 15 games')] = 'background-color: rgba(255, 255, 0, 0.3)'
    elif last_15 in ["10/15", "9/15", "8/15", "7/15", "6/15", "5/15", "4/15", "3/15", "2/15", "1/15", "0/15"]:
        styles[row.index.get_loc('Last 15 games')] = 'background-color: rgba(255, 0, 0, 0.3)'
    
    return styles

In [35]:
CLE_vs_HOU.style.apply(highlight_hitrate, axis=1)

,Player,Team,Stat,Line,Last 5 games,Last 10 games,Last 15 games
1,Caris LeVert,CLE,FG Attempted,8.500000,1/5,4/10,7/15
2,Caris LeVert,CLE,FG Made,4.000000,1/5,2/10,5/15
3,Caris LeVert,CLE,3-PT Attempted,4.000000,2/5,5/10,9/15
4,Caris LeVert,CLE,Offensive Rebounds,0.500000,3/5,4/10,6/15
5,Caris LeVert,CLE,Pts+Rebs+Asts,18.500000,2/5,3/10,5/15
6,Caris LeVert,CLE,Rebs+Asts,6.500000,3/5,4/10,4/15
7,Caris LeVert,CLE,Pts+Asts,14.500000,1/5,2/10,5/15
8,Caris LeVert,CLE,Blocked Shots,0.500000,4/5,6/10,8/15
9,Caris LeVert,CLE,Points,11.500000,1/5,2/10,5/15
10,Caris LeVert,CLE,Blks+Stls,1.500000,1/5,4/10,6/15


Parlay maker: 

In [ ]:
# merge all the dataframes into one
all_matchups = pd.concat(updated_matchup_dfs, ignore_index=True)

In [80]:
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors


# Custom colors
green_color = colors.Color(82 / 255, 183 / 255, 136 / 255)
yellow_color = colors.Color(250 / 255, 193 / 255, 114 / 255)
red_color = colors.Color(226 / 255, 91 / 255, 69 / 255)

# Function to determine background color based on the performance string
def get_background_color(performance):
    games_played, total_games = map(int, performance.split('/'))
    ratio = games_played / total_games

    if total_games == 5:
        if ratio >= 0.8:  # 4/5 or 5/5
            return green_color
        elif ratio >= 0.6:  # 3/5
            return yellow_color
        else:  # 2/5 or less
            return red_color

    elif total_games == 10:
        if ratio >= 0.8:  # 8/10, 9/10, or 10/10
            return green_color
        elif ratio >= 0.7:  # 7/10
            return yellow_color
        else:  # 6/10 or less
            return red_color

    elif total_games == 15:
        if ratio >= 0.8:  # 12/15, 13/15, 14/15, or 15/15
            return green_color
        elif ratio >= 0.733:  # 11/15
            return yellow_color
        else:  # 10/15 or less
            return red_color

    return colors.white  # Default color if none of the conditions match

# Create a PDF document
pdf = SimpleDocTemplate("CLE_vs_HOU Jan 22.pdf", pagesize=letter)

# Prepare data for the table
data = [CLE_vs_HOU.columns.tolist()] + CLE_vs_HOU.values.tolist()

# Create a table
table = Table(data)

# Add base styling
style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.gray),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
])

# Apply background color logic for specific columns
for i, row in enumerate(CLE_vs_HOU.itertuples(), start=1):
    last_5_color = get_background_color(row._5)
    last_10_color = get_background_color(row._6)
    last_15_color = get_background_color(row._7)

    style.add('BACKGROUND', (4, i), (4, i), last_5_color)
    style.add('BACKGROUND', (5, i), (5, i), last_10_color)
    style.add('BACKGROUND', (6, i), (6, i), last_15_color)

# Set the table style
table.setStyle(style)

# Build the PDF
pdf.build([table])
